<a href="https://colab.research.google.com/github/mladen12/git-test/blob/main/Character_Level_Language_Ivo_Andric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# ***Imports***



In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import math

# ***Data Preprocessing and Loading***



In [13]:
with open('ivo-andric-na-drini-cuprija.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [14]:
print("Lenght of dataset in chars: ", len(text))

Lenght of dataset in chars:  649893


In [15]:
print(text[:1000])



Иво Андрић

На Дрини ћуприја
I
Већим делом свога тока река Дрина протиче кроз тесне
гудуре између стрмих планина или кроз дубоке кањоне окомито одсечених обала. Само на неколико места речног тока
њене се обале проширују у отворене долине и стварају, било
на једној било на обе стране реке, жупне, делимично равне,
делимично таласасте пределе, подесне за обрађивање и насеља. Такво једно проширење настаје и овде, код Вишеграда, на
месту где Дрина избија у наглом завоју из дубоког и уског теснаца који стварају Буткове Стијене и Узавничке планине. Заокрет који ту прави Дрина необично је оштар а планине са обе
стране тако су стрме и толико ублизу да изгледају као затворен
масив из кога река извире право, као из мрког зида. Али ту се
планине одједном размичу у неправилан амфитеатар чији
промер на најширем месту није већи од петнаестак километара ваздушне линије.
На том месту где Дрина избија целом тежином своје
водене масе, зелене и запењене, из привидно затвореног склопа црних и стрмих пл

In [16]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(f"Each character without repetition: {vocab_size}.")


 !&'(),-.0123456789:;?ABCEGHIJKLMNORSUVWXZ_abcdefghiklmnoprstuvwxz«°»üčЂЈЊЋЏАБВГДЕЖЗИКЛМНОПРСТУФХЦЧШабвгдежзиклмнопрстуфхцчшђјљњћџ–
Each character without repetition: 133.


In [17]:
class EncoderDecoder:
    def __init__(self, chars):
        self.stoi = {ch: i for i, ch in enumerate(chars)}
        self.itos = {i: ch for i, ch in enumerate(chars)}

    def encode(self, s):
        return [self.stoi[c] for c in s]

    def decode(self, l):
        return ''.join([self.itos[i] for i in l])

encoder_decoder = EncoderDecoder(chars)
encoded_text = encoder_decoder.encode("Ivo Andric")
decoded_text = encoder_decoder.decode(encoded_text)

print(encoded_text)
print(decoded_text)

[30, 64, 58, 2, 24, 57, 48, 60, 53, 47]
Ivo Andric


In [18]:
data = torch.tensor(encoder_decoder.encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([649893]) torch.int64
tensor([  0,   0,   1,  86, 104, 115,   2,  78, 114, 106, 117, 110, 130,   0,
          0,  90, 102,   2,  82, 117, 110, 114, 110,   2, 130, 120, 116, 117,
        110, 127, 102,   0,   1,  30,   0,  80, 107, 130, 110, 113,   2, 106,
        107, 112, 115, 113,   2, 118, 104, 115, 105, 102,   2, 119, 115, 111,
        102,   2, 117, 107, 111, 102,   2,  82, 117, 110, 114, 102,   2, 116,
        117, 115, 119, 110, 124, 107,   2, 111, 117, 115, 109,   2, 119, 107,
        118, 114, 107,   0, 105, 120, 106, 120, 117, 107,   2, 110, 109, 113,
        107, 126, 120,   2, 118, 119, 117, 113, 110, 122,   2, 116, 112, 102,
        114, 110, 114, 102,   2, 110, 112, 110,   2, 111, 117, 115, 109,   2,
        106, 120, 103, 115, 111, 107,   2, 111, 102, 129, 115, 114, 107,   2,
        115, 111, 115, 113, 110, 119, 115,   2, 115, 106, 118, 107, 124, 107,
        114, 110, 122,   2, 115, 103, 102, 112, 102,  10,   2,  94, 102, 113,
        115,   2, 114, 102,   2

# ***Spliting the data***


In [19]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [20]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# ***hyperparameters***

In [23]:
batch_size = 32
block_size = 64
max_iters = 30000
eval_interval = 1000
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 8
n_layer = 6
dropout = 0.2

# ***Decoder components with training, evaluation, and generation***

In [24]:
# Define model components
class MultiHeadAttention(nn.Module):
    """ Multi-head self-attention module """

    def __init__(self, num_heads, head_size, n_embd):
        super().__init__()
        self.num_heads = num_heads
        self.head_size = head_size
        self.n_embd = n_embd

        # Linear projections for queries, keys, and values for all heads
        self.query = nn.Linear(n_embd, num_heads * head_size, bias=False)
        self.key = nn.Linear(n_embd, num_heads * head_size, bias=False)
        self.value = nn.Linear(n_embd, num_heads * head_size, bias=False)

        self.proj = nn.Linear(num_heads * head_size, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.size()

        queries = self.query(x).view(B, T, self.num_heads, self.head_size).transpose(1, 2)
        keys = self.key(x).view(B, T, self.num_heads, self.head_size).transpose(1, 2)
        values = self.value(x).view(B, T, self.num_heads, self.head_size).transpose(1, 2)

        attention_scores = torch.matmul(queries, keys.transpose(-2, -1)) / math.sqrt(self.head_size)
        mask = torch.tril(torch.ones(T, T, device=x.device)).view(1, 1, T, T)
        attention_scores = attention_scores.masked_fill(mask == 0, float('-inf'))
        attention_weights = F.softmax(attention_scores, dim=-1)
        attention_weights = self.dropout(attention_weights)
        attention_output = torch.matmul(attention_weights, values)

        attention_output = attention_output.transpose(1, 2).contiguous().view(B, T, -1)  # (B, T, num_heads * head_size)
        attention_output = self.proj(attention_output)  # (B, T, C)

        return attention_output

class FeedForward(nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size, n_embd)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-5)

for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_text = encoder_decoder.decode(m.generate(context, max_new_tokens=2000)[0].tolist())
print(generated_text)

0.320133 M parameters
step 0: train loss 5.0491, val loss 5.0461
step 1000: train loss 2.2072, val loss 2.2398
step 2000: train loss 2.0038, val loss 2.0415
step 3000: train loss 1.8819, val loss 1.9407
step 4000: train loss 1.8123, val loss 1.8873
step 5000: train loss 1.7671, val loss 1.8457
step 6000: train loss 1.7350, val loss 1.8272
step 7000: train loss 1.6968, val loss 1.7940
step 8000: train loss 1.6781, val loss 1.7799
step 9000: train loss 1.6630, val loss 1.7715
step 10000: train loss 1.6457, val loss 1.7649
step 11000: train loss 1.6282, val loss 1.7462
step 12000: train loss 1.6238, val loss 1.7387
step 13000: train loss 1.6121, val loss 1.7337
step 14000: train loss 1.6032, val loss 1.7220
step 15000: train loss 1.5874, val loss 1.7205
step 16000: train loss 1.5794, val loss 1.7202
step 17000: train loss 1.5755, val loss 1.7186
step 18000: train loss 1.5733, val loss 1.7096
step 19000: train loss 1.5652, val loss 1.7038
step 20000: train loss 1.5580, val loss 1.7058
step